In [2]:
print("ok")

ok


In [3]:
%pwd

'd:\\Building-Stateful-Multi-AI-Agents-With-LangGraph\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'd:\\Building-Stateful-Multi-AI-Agents-With-LangGraph'

In [3]:
from dotenv import load_dotenv  # Import the `load_dotenv` function to load environment variables from a `.env` file.
import os  # Import the `os` module to interact with the operating system, specifically to access environment variables.
import cassio  # type: ignore # Import the `cassio` module to work with the Astra DB database.
# Load environment variables from a `.env` file and make them available in the environment.
load_dotenv()
# Retrieve the application token for Astra DB from the environment variables.
ASTRA_DB_APPLICATION_TOKEN = os.getenv('ASTRA_DB_APPLICATION_TOKEN')  
# Retrieve the database ID for Astra DB from the environment variables.
ASTRA_DB_ID = os.getenv('ASTRA_DB_ID')
# Initialize the connection to the Cassio database using the application token and database ID.
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [6]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_pdf(file_path, chunk_size, chunk_overlap):
    """Load a single PDF and split into chunks with custom separators."""
    
    # Load the PDF file
    loader = PyPDFLoader(file_path)
    documents = loader.load()

    # Define custom separators for splitting text
    separators = [
        "\n\n",  # Paragraph break
        "\n",    # Line break
        " ",     # Space
        ".",     # Full stop
        ",",     # Comma
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        ""        # Empty separator
    ]

    # Create a text splitter with the given chunk size, overlap, and separators
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,
        separators=separators
    )
    
    # Split the documents into smaller chunks
    return text_splitter.split_documents(documents)


In [7]:
file_path = r"D:\Building-Stateful-Multi-AI-Agents-With-LangGraph\data\ThaiRecipes.pdf"  # Path to the specific PDF file

# Call the function with the file path
doc_chunks = load_and_split_pdf(file_path, chunk_size=500, chunk_overlap=10)


In [8]:
doc_chunks

[Document(metadata={'source': 'D:\\Building-Stateful-Multi-AI-Agents-With-LangGraph\\data\\ThaiRecipes.pdf', 'page': 0}, page_content='C O O K B O O K\nwww.thaiselect.com www.thaiselect.com'),
 Document(metadata={'source': 'D:\\Building-Stateful-Multi-AI-Agents-With-LangGraph\\data\\ThaiRecipes.pdf', 'page': 1}, page_content='Thai SELECT Exotic, Healthy, Delicious Dining-Thai Style\nThai food has rapidly grown in popularity among casual diners and gourmets alike, \nearning it a status as one of the world’s most popular cuisines. While most people \nthink of spicy meals laced with chilies as the predominant factor in Thai food, this is far \nfrom the truth. \nIn all Thai dishes, there must always be a balanced harmony of flavors. Whether in a'),
 Document(metadata={'source': 'D:\\Building-Stateful-Multi-AI-Agents-With-LangGraph\\data\\ThaiRecipes.pdf', 'page': 1}, page_content='rich or fragrant Thai curry, spicy soup, savory salad, or sweet dessert, the competing \nand complementing fla

In [2]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Get the API key from environment variables
api_key = os.getenv('google_api_key')
os.environ["GOOGLE_API_KEY"] = api_key

# Download the embeddings from Hugging Face
def get_google_embeddings():
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    return embeddings

d:\Building-Stateful-Multi-AI-Agents-With-LangGraph\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
embeddings = get_google_embeddings()

In [8]:
from langchain.vectorstores.cassandra import Cassandra
astra_vector_store=Cassandra(
    embedding=embeddings,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None

)

In [18]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
astra_vector_store.add_documents(doc_chunks)
print("Inserted %i headlines." % len(doc_chunks))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 45 headlines.


In [9]:
retriever = astra_vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.5}
)

In [10]:
### Define Router

# Import necessary modules
from typing import Literal  # For defining fields with restricted literal values  # For creating user prompt templates
from langchain_core.pydantic_v1 import BaseModel, Field  # For data validation and defining model fields with metadata


# Define a Pydantic model for routing queries
class RouteQuery(BaseModel):
    """
    Route a user query to the most relevant datasource.
    This model determines whether a user query should be processed 
    by duckduckgo-search or a vectorstore.
    """
    datasource: Literal["vectorstore", "duckduckgo-search"] = Field(
        ...,  # Indicates this field is required
        description="Given a user question, choose to route it to duckduckgo-search or a vectorstore.",
    )

d:\Building-Stateful-Multi-AI-Agents-With-LangGraph\myenv\lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
import os

# Initialize the ChatGoogleGenerativeAI model with convert_system_message_to_human set to True
llm = ChatGoogleGenerativeAI(
    model='gemini-2.0-flash-exp',  # Specify the model version
    google_api_key=os.getenv('GOOGLE_API_KEY'),
    temperature=0.1,  # Controls randomness; higher values yield more creative responses
    max_output_tokens=30,  # Limits the length of the generated response
    #top_k=50,  # Number of top tokens to consider for each generation step
    #top_p=0.95,  # Cumulative probability for token selection
    convert_system_message_to_human=True  # Merge SystemMessage into HumanMessage
)

# Configure the structured output for routing
structured_llm_router = llm.with_structured_output(RouteQuery)

# Define the system message for routing logic
system_message = """
You are ThaiRecipes, a helpful and concise assistant specializing in Thai cuisine.

For inquiries related to Thai recipes, ingredients, cooking techniques, or cultural food traditions, use the vectorstore to retrieve the most relevant documents.

For all other topics, perform a DuckDuckGo search to find the best available information.

Answer only questions related to Thai cuisine yourself. Ensure users always receive the most accurate, reliable, and helpful information based on their queries and outformat in table .
"""

# Create the routing prompt
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", "{question}"),
    ]
)

# Combine the routing prompt with the structured LLM
question_router = route_prompt | structured_llm_router

# Test the routing system with sample questions
#print(question_router.invoke({"question": "Who is Shahrukh Khan?"}))
#print(question_router.invoke({"question": "Suresh beekhani"}))


In [12]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()



In [13]:
from typing import List
from typing_extensions import TypedDict

# Define the GraphState class using TypedDict
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: A string representing a question.
        generation: A string representing an LLM generation.
        documents: A list of strings representing related documents.
    """
    question: str
    generation: str
    documents: List[str]


In [14]:
# Import the Document class from langchain.schema, which represents a document object.
from langchain.schema import Document

# Define a function 'retrieve' that retrieves documents based on the current state
def retrieve(state):
    """
    Retrieve documents based on the provided question in the state.

    Args:
        state (dict): The current graph state, which contains the key "question".

    Returns:
        state (dict): A new state with an added key "documents", which contains the retrieved documents.
    """
    # Print a message to indicate the start of the retrieve process
    #print("---RETRIEVE---")

    # Extract the 'question' from the state
    question = state["question"]

    # Call the retriever (external tool/function) to fetch documents related to the question
    documents = retriever.invoke(question)

    # Return the updated state, including the retrieved documents and the original question
    return {"documents": documents, "question": question}


# Define a function 'DuckDuckGo_search' that performs a search using DuckDuckGo based on the rephrased question
def DuckDuckGo_search(state):
    """
    Perform a search using DuckDuckGo based on the re-phrased question.

    Args:
        state (dict): The current graph state, which contains the key "question".

    Returns:
        state (dict): Updates the "documents" key with the results from the DuckDuckGo search.
    """
    # Print messages for debugging purposes to indicate the start of the search process
    #print("---wikipedia---")
    #print("---HELLO--")

    # Extract the 'question' from the state
    question = state["question"]
    #print(question)

    # Perform the search using DuckDuckGo (search.invoke is assumed to be an external search function)
    docs = search.invoke({"query": question})

    # Assign the search results to a variable 'DuckDuckGo_results'
    DuckDuckGo_results = docs

    # Convert the search results into a Document object, as expected by the subsequent processing steps
    DuckDuckGo_results = Document(page_content=DuckDuckGo_results)

    # Return the updated state, including the search results (wrapped in a Document) and the original question
    return {"documents": DuckDuckGo_results, "question": question}


In [15]:
### Edges ###


def route_question(state):
    """
    Route question to wiki search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    #print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "duckduckgo-search":
        #print("---ROUTE QUESTION TO Wiki SEARCH---")
        return "duckduckgo-search"
    elif source.datasource == "vectorstore":
        #print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

In [27]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)
# Define the nodes
workflow.add_node("duckduckgo-search", DuckDuckGo_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve

# Build graph
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "duckduckgo-search": "duckduckgo-search",
        "vectorstore": "retrieve",
    },
)
workflow.add_edge( "retrieve", END)
workflow.add_edge( "duckduckgo-search", END)

# Compile
app = workflow.compile()

In [32]:
# Define the function and run the app
def run_app(question):
    result = app.invoke({"question": question})
    return result  # Return the result from the app invocation


In [44]:
question = "HOW make tae"
print(run_app(question))

d:\Building-Stateful-Multi-AI-Agents-With-LangGraph\myenv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'question': 'HOW make tae', 'documents': [Document(id='112fb431e6e845669cbb35573e27a5b1', metadata={'page': '3.0', 'source': 'D:\\Building-Stateful-Multi-AI-Agents-With-LangGraph\\data\\ThaiRecipes.pdf'}, page_content='finely chopped\n1 tbsp roasted peanuts\n50 grams palm sugar\n10 grams dried shrimps\n50 grams firm tofu, thinly sliced\n150 grams thin noodles\n100 grams bean sprouts\n20 grams chives, cut in one inch\nlengths\n1 egg\n1/4 lime\n25\nMin'), Document(id='e245ba0e182a49ae93f6a9ec8489dd72', metadata={'page': '11.0', 'source': 'D:\\Building-Stateful-Multi-AI-Agents-With-LangGraph\\data\\ThaiRecipes.pdf'}, page_content='2.Add sugar and continue boiling until well dissolved.\n3. Add coconut flesh. Stir well. Bring to a boil once again\nand turn off the heat. 4. Mix the coconut cream and salt \ntogether and place over low heat. Stir regularly to \nprevent the coconut cream from separating. When the \ncoconut cream is well heated and salt is thoroughly \ndissolved, remove from th